# Project Milestone 2 Notebook
## Do Americans prefer beers with a higher alcohol content (ABV) than Europeans?
- Is it linked to the beer style? Do they generally prefer beer styles that have a higher ABV? (Grouping + micro/macro averages)
- Has it evolved between 2000 and 2017? (Time series analysis + maybe regression)
- Can we map American States to European countries? (Graph/network algorithms)

## 0. Imports and global variables

In [283]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from load_data import load_gzip_txt_data

## 1. Preprocessing of the data:

#### 1.0 Split users between North America and Europe:

Define European countries:

In [284]:
european_countries = [
    "Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus",
    "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus",
    "Czech Republic", "Denmark", "England", "Estonia", "Finland", "France", "Georgia",
    "Germany", "Gibraltar", "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Jersey", "Kazakhstan",
    "Kosovo", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg", "Malta",
    "Moldova", "Monaco", "Montenegro", "Netherlands", "Northern Ireland", "Macedonia",
    "Norway", "Poland", "Portugal", "Romania", "Russia", "San Marino", "Scotland", "Serbia",
    "Slovakia", "Slovak Republic", "Slovenia", "Spain", "Sweden", "Switzerland", "Turkey",
    "Ukraine", "Vatican City", "Wales"
]

# /!\ Slovakia in RateBeer and Slovak Republic in BeerAdvocate /!\

Load users files and classify if European, North American or neither:

In [285]:
def get_na_or_eu(row):
    """
    In a Dataframe with a column "location", returns whether the location 
    is in Europe or North America.
    
    :param row: pandas.Dataframe row with an attribute "location".
    :return: str, can be "NA", "EU" or "Other".
    """
    location = row['location']
    if location in european_countries:
        return 'EU'
    elif (location == 'Canada') or ("United States" in str(location)):
        return 'NA'
    else:
        return 'Other'


ba_users_df = (pd.read_csv("./data/BeerAdvocate/users.csv").drop(
    columns=['nbr_ratings', 'nbr_reviews', 'user_name', 'joined'])
               .dropna())
ba_users_df['eu_na'] = ba_users_df.apply(get_na_or_eu, axis=1)

rb_users_df = (pd.read_csv("./data/RateBeer/users.csv").drop(
    columns=['nbr_ratings', 'user_name', 'joined'])
               .dropna())
rb_users_df['eu_na'] = rb_users_df.apply(get_na_or_eu, axis=1)

print("BeerAdvocate:")
display(ba_users_df)
print("RateBeer:")
display(rb_users_df)

BeerAdvocate


,user_id,location,eu_na
0,nmann08.184925,"United States, Washington",NA
1,stjamesgate.163714,"United States, New York",NA
2,mdagnew.19527,Northern Ireland,EU
3,helloloser12345.10867,Northern Ireland,EU
4,cypressbob.3708,Northern Ireland,EU
...,...,...,...
153698,eturchick.374415,"United States, California",NA
153699,everman.532342,"United States, California",NA
153700,justin0001.352175,"United States, California",NA
153702,joetex.800347,"United States, California",NA


RateBeer


,user_id,location,eu_na
0,175852,Poland,EU
1,442761,Spain,EU
2,288889,Poland,EU
3,250510,Spain,EU
4,122778,Germany,EU
...,...,...,...
70167,181614,Norway,EU
70169,134893,England,EU
70170,327816,Belgium,EU
70172,82020,Slovenia,EU


Let's analyze the repartition of NA/EU/Other users:

In [286]:
print("BeerAdvocate:")
display(ba_users_df['eu_na'].value_counts())
print("RateBeer:")
display(rb_users_df['eu_na'].value_counts())

BeerAdvocate:


eu_na
NA       116547
EU         3944
Other      1934
Name: count, dtype: int64


RateBeer:


eu_na
NA       30110
EU       16156
Other     4326
Name: count, dtype: int64

Clearly, the vast majority of users comes from North America, followed by Europe especially in the BeerAdvocate dataset. In the following analysis, we will only consider NA and EU users.


In [287]:
ba_users_df = ba_users_df.query("eu_na != 'Other'")
rb_users_df = rb_users_df.query("eu_na != 'Other'")

#### 1.1 Load the ratings:

Load the ratings files:

In [288]:
ba_ratings_df = load_gzip_txt_data("./data/BeerAdvocate/ratings.txt.gz",
                                   ["user_id", "date", "abv", "style", "rating", "appearance", "aroma", "palate",
                                    "taste", "overall"],
                                   max_entries=100000)
rb_ratings_df = load_gzip_txt_data("./data/RateBeer/ratings.txt.gz",
                                   ["user_id", "date", "abv", "style", "rating", "appearance", "aroma", "palate",
                                    "taste", "overall"],
                                   max_entries=100000)

Loading data from:  ./data/BeerAdvocate/ratings.txt.gz


1799996it [00:02, 780436.58it/s]


Loading data from:  ./data/RateBeer/ratings.txt.gz


1699997it [00:02, 784358.67it/s]


Cast the columns to meaningful types:

In [289]:
ba_ratings_df = ba_ratings_df.astype({
    'user_id': 'str',
    'date': 'int64',
    'abv': 'float32',
    'rating': 'float32'})

rb_ratings_df = rb_ratings_df.astype({
    'user_id': 'int64',
    'date': 'int64',
    'abv': 'float32',
    'rating': 'float32'})

# Convert the dates to monthly periods (useful for plotting):
ba_ratings_df['date'] = pd.to_datetime(ba_ratings_df['date'], unit='s').dt.to_period('m').astype('datetime64[M]')
rb_ratings_df['date'] = pd.to_datetime(rb_ratings_df['date'], unit='s').dt.to_period('m').astype('datetime64[M]')

#### 1.2 Group similar beer styles into broader groups

Defined the broad groups using https://www.beeradvocate.com/beer/styles/, so we can just load the csv file containing the matching specific type -> general type:

In [290]:
beer_styles_matching = pd.read_csv("./data/additional/styles_matching.csv", index_col='specific')['general'].to_dict()
print(f"General styles:\n{set(beer_styles_matching.values())}")

General styles:
{'Pale Ales', 'India Pale Ales', 'Specialty Beers', 'Stouts', 'Wild/Sour Beers', 'Wheat Beers', 'Saké', 'Porters', 'Dark Lagers', 'Dark Ales', 'Strong Ales', 'Hybrid Beers', 'Pale Lagers', 'Bocks', 'Brown Ales'}


Note: we don't want to take the Saké into account as it is not a beer!

Replace the specific beer style by the general style (and drop Saké ratings):

In [291]:
def specific_style_to_general(row):
    return beer_styles_matching[row['style']]

ba_ratings_df['style'] = ba_ratings_df.apply(specific_style_to_general, axis=1)
rb_ratings_df['style'] = rb_ratings_df.apply(specific_style_to_general, axis=1)

# Drop Saké
ba_ratings_df = ba_ratings_df.query("style != 'Saké'")
rb_ratings_df = rb_ratings_df.query("style != 'Saké'")

In [292]:
print("BeerAdvocate:")
display(ba_ratings_df.sample(10))
print("RateBeer:")
display(rb_ratings_df.sample(10))

BeerAdvocate:


,user_id,date,abv,style,rating,appearance,aroma,palate,taste,overall
76057,rodbeermunch.1073239,2016-08-01,5.0,Stouts,3.69,4.25,3.25,4.0,3.75,3.75
30961,jamland.731672,2013-06-01,5.2,Dark Ales,3.75,nan,nan,nan,nan,nan
80154,ferrari29.320392,2011-06-01,5.0,Stouts,3.73,4.5,4.0,4.0,3.5,3.5
17069,pootz.916,2017-07-01,6.2,India Pale Ales,3.69,4.0,3.75,3.0,3.75,3.75
47199,hopdog.9038,2008-12-01,5.6,India Pale Ales,3.70,3.5,3.5,3.5,4.0,3.5
62223,dj-hophead.992772,2016-08-01,7.0,India Pale Ales,4.21,4.0,4.25,4.0,4.25,4.25
12824,davy_b.959698,2015-03-01,6.6,Dark Ales,4.21,5.0,4.0,4.5,4.0,4.5
10297,weissboy.652453,2012-01-01,7.1,Strong Ales,4.00,nan,nan,nan,nan,nan
67961,vec2267.117028,2011-01-01,7.0,Stouts,4.63,4.5,4.0,4.0,5.0,5.0
56842,fisherman34.19192,2006-09-01,4.7,Pale Ales,2.82,3.0,3.5,2.0,2.5,3.0


RateBeer:


,user_id,date,abv,style,rating,appearance,aroma,palate,taste,overall
98030,274979,2015-05-01,5.80,Wild/Sour Beers,3.7,3,7,3,8,16
25201,15284,2015-02-01,5.20,Dark Lagers,2.9,3,5,3,6,12
54485,23644,2006-06-01,3.70,Pale Lagers,1.0,1,1,1,2,5
24113,305742,2014-03-01,6.20,Dark Lagers,4.4,5,9,4,9,17
69660,364257,2016-07-01,7.10,Pale Lagers,1.5,3,2,2,3,5
10626,549,2006-08-01,5.30,Pale Lagers,1.0,1,2,1,2,4
91007,8067,2014-03-01,7.00,Hybrid Beers,3.4,3,7,3,7,14
85614,8067,2014-03-01,10.00,Stouts,3.7,4,7,3,7,16
72392,283370,2017-01-01,5.10,Wild/Sour Beers,2.1,3,6,2,3,7
49797,9380,2005-03-01,4.55,Pale Lagers,1.6,2,5,1,1,7


#### 1.3 Merge the users with the ratings

In [293]:
ba_df = ba_ratings_df.merge(ba_users_df, on='user_id').drop(columns=['user_id'])
rb_df = rb_ratings_df.merge(rb_users_df, on='user_id').drop(columns=['user_id'])

In [294]:
print("BeerAdvocate:")
display(ba_df)
print("RateBeer:")
display(rb_df)

BeerAdvocate:


,date,abv,style,rating,appearance,aroma,palate,taste,overall,location,eu_na
0,2015-08-01,4.5,Pale Lagers,2.88,3.25,2.75,3.25,2.75,3.0,"United States, Washington",NA
1,2015-11-01,5.0,Pale Ales,3.56,4.0,3.5,3.75,3.5,3.5,"United States, Washington",NA
2,2011-11-01,8.0,Strong Ales,4.50,nan,nan,nan,nan,nan,"United States, Washington",NA
3,2013-05-01,10.5,Strong Ales,3.75,nan,nan,nan,nan,nan,"United States, Washington",NA
4,2015-01-01,6.0,Porters,3.81,4.0,3.5,3.75,4.0,3.75,"United States, Washington",NA
...,...,...,...,...,...,...,...,...,...,...,...
91537,2012-06-01,4.7,Pale Ales,4.27,4.0,4.5,4.5,4.0,4.5,"United States, Georgia",NA
91538,2012-06-01,4.7,Pale Ales,3.25,nan,nan,nan,nan,nan,"United States, Illinois",NA
91539,2012-06-01,4.7,Pale Ales,3.00,nan,nan,nan,nan,nan,"United States, Pennsylvania",NA
91540,2012-05-01,4.7,Pale Ales,3.50,nan,nan,nan,nan,nan,"United States, Florida",NA


RateBeer:


,date,abv,style,rating,appearance,aroma,palate,taste,overall,location,eu_na
0,2016-04-01,5.0,Pale Lagers,2.0,2,4,2,4,8,Poland,EU
1,2015-05-01,5.2,Pale Lagers,1.7,2,4,2,3,6,Poland,EU
2,2015-04-01,4.5,Pale Lagers,1.9,3,4,2,4,6,Poland,EU
3,2015-04-01,5.3,Brown Ales,3.7,4,7,4,8,14,Poland,EU
4,2015-11-01,6.1,Brown Ales,2.5,4,6,3,5,7,Poland,EU
...,...,...,...,...,...,...,...,...,...,...,...
89706,2014-02-01,6.0,Porters,4.5,5,9,5,8,18,Canada,NA
89707,2015-08-01,6.0,Porters,3.7,4,7,3,7,16,Spain,EU
89708,2015-06-01,6.0,Porters,4.4,4,9,5,8,18,France,EU
89709,2016-12-01,8.5,India Pale Ales,4.0,4,8,3,8,17,Canada,NA


## 2. General Analysis 

In [295]:
# TODO: Plot some stuff using what we have here: NA or EU, ABV values, 14 possible styles, sensory ratings (can be nan), and precise location (maybe keep this for later).

# TODO: Do we want to use a bit of NLP ? Not sure how to do it in a meaningful way...

# TODO: /!\ Please plot with seaborn if it can be simpler (mainly for automatic CI) I beg you ^^